1. IMPORT LIBRARY YANG DIPERLUKAN

In [ ]:
import os
import pandas as pd

2. PATH FOLDER DATA

In [ ]:
# Path utama folder Data Fitur PPG
base_folder = 'data uji'  # Ganti dengan path folder utama Anda

3. FITUR YANG AKAN DIPAKAI, DAN KOLOM FITUR YANG LAIN AKAN DIHAPUS

In [ ]:
# Fitur yang akan digunakan
selected_features = ['HR_mean', 'SDNN', 'RMSSD', 'pNN50']

4. MEMPROSES DATA CSV

In [ ]:
# Fungsi untuk memproses CSV
def process_csv(file_path):
    # Baca file CSV
    data = pd.read_csv(file_path)
    
    # Konversi fitur menjadi numerik dan ganti error dengan NaN
    for feature in selected_features:
        data[feature] = pd.to_numeric(data[feature], errors='coerce')
    
    # Drop baris dengan nilai NaN
    data_selected = data[selected_features].dropna()
    
    # Tambahkan validasi untuk memastikan data valid
    if data_selected.empty:
        print(f"Peringatan: Tidak ada data valid setelah membersihkan file {file_path}")
        return pd.DataFrame()  # Return DataFrame kosong jika tidak ada data valid
    
    return data_selected

5. MEMPROSES CSV SESUAI DENGAN PATH FOLDER DAN AKAN MENYIMPAN FILE BARU DARI DATA YANG SUDAH DI SELEKSI FITUR

In [ ]:
# Iterasi setiap folder partisipan
for participant in os.listdir(base_folder):
    participant_path = os.path.join(base_folder, participant)
    if os.path.isdir(participant_path):
        # Iterasi setiap sesi
        for session in os.listdir(participant_path):
            session_path = os.path.join(participant_path, session, "features.csv")
            if os.path.exists(session_path):
                print(f"Memproses file: {session_path}")
                
                # Proses file CSV
                processed_data = process_csv(session_path)
                
                # Path untuk menyimpan file hasil
                output_file = session_path.replace('.csv', '_processed.csv')
                
                # Simpan data yang sudah diproses
                processed_data.to_csv(output_file, index=False)
                print(f"File hasil diproses disimpan di: {output_file}")

6. MENGGABUNGKAN DATA SETIAP SESI DARI PER SUBJEK DAN AKAN DISIMPAN DALAM FOLDER BARU

In [ ]:
# Path utama folder Data Fitur PPG
base_folder = 'Data Fitur PPG'  # Ganti dengan path folder utama Anda
output_folder = 'Gabungan Sesi PPG'  # Folder untuk menyimpan hasil gabungan
os.makedirs(output_folder, exist_ok=True)

# Daftar nama sesi
session_names = ["sesi 1", "sesi 2", "sesi 3", "sesi 4"]

# Dictionary untuk menyimpan data dari setiap sesi
session_data = {session: [] for session in session_names}

# Iterasi setiap folder partisipan
for participant in os.listdir(base_folder):
    participant_path = os.path.join(base_folder, participant)
    if os.path.isdir(participant_path):
        # Iterasi setiap sesi
        for session in session_names:
            session_file = os.path.join(participant_path, session, "features.csv")  # Gunakan file asli
            if os.path.exists(session_file):
                print(f"Membaca file: {session_file}")
                # Baca file CSV
                data = pd.read_csv(session_file)
                # Tambahkan data ke list sesuai sesi
                session_data[session].append(data)

# Gabungkan data untuk setiap sesi dan simpan ke file baru
for session, data_list in session_data.items():
    if data_list:  # Jika ada data untuk sesi ini
        combined_data = pd.concat(data_list, ignore_index=True)
        output_file = os.path.join(output_folder, f"{session}_combined.csv")
        combined_data.to_csv(output_file, index=False)
        print(f"File gabungan untuk {session} disimpan di: {output_file}")
